# Clustering Crypto

In [67]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [68]:
# Load the crypto_data.csv dataset.
file = "crypto_data.csv"
crypto_df = pd.read_csv(file, index_col = 0)
crypto_df.shape

(1252, 6)

In [69]:
crypto_df.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [ ]:
#No need to drop because already set as index

#crypto_df = crypto_df.drop(columns = "Unnamed: 0")
#crypto_df.head()

In [70]:
crypto_df["IsTrading"].value_counts()

True     1144
False     108
Name: IsTrading, dtype: int64

In [71]:
crypto_df = crypto_df[(crypto_df["IsTrading"] == True)]

In [72]:
crypto_df.shape

(1144, 6)

In [73]:
crypto_df["Algorithm"].value_counts()

Scrypt                 394
X11                    182
SHA-256                121
X13                     54
PoS                     42
                      ... 
CryptoNight Heavy X      1
XG Hash                  1
Dagger-Hashimoto         1
QUAIT                    1
Exosis                   1
Name: Algorithm, Length: 89, dtype: int64

In [74]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df["Algorithm"].isnull().sum()
#Keeping all

0

In [75]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(columns = "IsTrading")
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [76]:
crypto_df.shape

(1144, 5)

In [77]:
# Remove rows that have at least 1 null value.
crypto_df.dropna(inplace = True)
crypto_df.shape

(685, 5)

In [78]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[(crypto_df["TotalCoinsMined"] != 0)]
crypto_df.shape

(533, 5)

In [79]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coin_name_df = pd.DataFrame(crypto_df["CoinName"])
coin_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [80]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns = "CoinName")
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [81]:
# Use get_dummies() to create variables for text features.
X_df = pd.get_dummies(crypto_df, columns = ["Algorithm","ProofType"])

In [82]:
X_df.shape

(533, 100)

In [83]:
X_df.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X_df)

In [85]:
X_scaled

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [86]:
# Using PCA to reduce dimension to three principal components.

# Initialize PCA model
pca = PCA(n_components=3)

In [87]:
# Get two principal components for the iris data.
X_pca = pca.fit_transform(X_scaled)

In [88]:
# Create a DataFrame with the three principal components.
crypto_df_pca = pd.DataFrame(
    data = X_pca, columns = ["PC 1", "PC 2", "PC 3"], index = crypto_df.index
)
crypto_df_pca.head(10)

,PC 1,PC 2,PC 3
42,-0.331123,1.068436,-0.634976
404,-0.314412,1.068727,-0.635278
1337,2.306674,1.669559,-0.694434
BTC,-0.146229,-1.336690,0.216905
ETH,-0.153555,-2.057228,0.462273
LTC,-0.161843,-1.098256,-0.016104
DASH,-0.385246,1.270979,-0.568322
XMR,-0.150300,-2.229844,0.486469
ETC,-0.151991,-2.057307,0.462261
ZEC,-0.147010,-2.152188,0.592227


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [89]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

In [90]:
# Looking for the best K
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_df_pca)
    inertia.append(km.inertia_)

C:\Users\johnj\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



In [91]:
# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [92]:
# Initializing model with K = 4 (based off elbow curve graph)
model = KMeans(n_clusters=4, random_state=5)
model

# Fit the model
model.fit(crypto_df_pca)

# Predict clusters
predictions = model.predict(crypto_df_pca)

# Add the predicted class column to the dataframe
crypto_df_pca["Class"] = model.labels_
crypto_df_pca.head()

,PC 1,PC 2,PC 3,Class
42,-0.331123,1.068436,-0.634976,0
404,-0.314412,1.068727,-0.635278,0
1337,2.306674,1.669559,-0.694434,0
BTC,-0.146229,-1.336690,0.216905,1
ETH,-0.153555,-2.057228,0.462273,1


In [93]:
predictions

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [94]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = crypto_df.join([crypto_df_pca["PC 1"],crypto_df_pca["PC 2"], crypto_df_pca["PC 3"]], how = 'inner')


#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(coin_name_df, how = 'inner')

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df = clustered_df.join(crypto_df_pca["Class"], how = 'inner')

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.331123,1.068436,-0.634976,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.314412,1.068727,-0.635278,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.306674,1.669559,-0.694434,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.146229,-1.336690,0.216905,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.153555,-2.057228,0.462273,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.161843,-1.098256,-0.016104,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.385246,1.270979,-0.568322,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150300,-2.229844,0.486469,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.151991,-2.057307,0.462261,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.147010,-2.152188,0.592227,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [95]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    width=800,
    hover_name = "CoinName",
    hover_data = ["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [99]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [105]:
# Print the total number of tradable cryptocurrencies.
count = len(clustered_df.index)
f"There are {count} tradable cryptocurrencies."

'There are 533 tradable cryptocurrencies.'

In [145]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
coin_data_df = clustered_df.drop(['CoinName','Algorithm','ProofType','Class','PC 1','PC 2','PC 3'], axis = 1)
coin_data_df = coin_data_df[['TotalCoinSupply','TotalCoinsMined']]
coin_data_df.head()

,TotalCoinSupply,TotalCoinsMined
42,42,4.199995e+01
404,532000000,1.055185e+09
1337,314159265359,2.927942e+10
BTC,21000000,1.792718e+07
ETH,0,1.076842e+08


In [157]:
scaler = MinMaxScaler()
coin_data_scaled = scaler.fit_transform(coin_data_df)
coin_data_scaled

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [1.40022261e-03, 6.92655266e-03],
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03]])

In [210]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
cds_df = pd.DataFrame(coin_data_scaled, index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
cds_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
cds_df["Class"] = clustered_df["Class"]

cds_df.head(10)

,0,1,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141593e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,1
ETH,0.000000e+00,0.006050,Ethereum,1
LTC,8.400000e-05,0.006006,Litecoin,1
DASH,2.200000e-05,0.005951,Dash,0
XMR,0.000000e+00,0.005960,Monero,1
ETC,2.100000e-04,0.006056,Ethereum Classic,1
ZEC,2.100000e-05,0.005950,ZCash,1


In [213]:
cds_df = cds_df.rename(columns = {0:"TotalCoinSupply",1:"TotalCoinsMined"})
cds_df.head()

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,0
404,5.320000e-04,0.007002,404Coin,0
1337,3.141593e-01,0.035342,EliteCoin,0
BTC,2.100000e-05,0.005960,Bitcoin,1
ETH,0.000000e+00,0.006050,Ethereum,1


In [215]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
cds_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    by="Class",
    hover_cols = "CoinName"
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)